In [1]:
# Install required libraries
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


In [2]:
import torch

# Clear GPU cache
torch.cuda.empty_cache()

In [3]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.modeling_utils import unwrap_model
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from collections import Counter
from typing import Optional

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load the TR-News dataset
dataset = load_dataset("batubayk/TR-News")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/614 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/686M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/36.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/277573 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14610 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15379 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 277573
    })
    validation: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 14610
    })
    test: Dataset({
        features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
        num_rows: 15379
    })
})


In [6]:
def clean_and_group_categories(dataset, min_samples=10, groupings=None):
    # Default groupings
    if groupings is None:
        groupings = {
            'Spor': ['spor', 'Basketbol', 'Futbol', 'Voleybol', '2016 Avrupa Şampiyonası', 'EURO 2016', 'Tenis', 'Motor Sporları', 'Olimpiyat', 'Spor', 'basketbol', 'futbol', 'diger_sporlar'],
            'Ekonomi': ['Ekonomi', 'Döviz', 'Para', 'Makro Ekonomi', 'ekonomi'],
            'Dünya': ['Dünya', 'dunya'],
            'Kültür-Sanat': ['Kültür-Sanat', 'Sanat', 'Müzik', 'kultur-sanat'],
            'Yaşam': ['Yaşam', 'yasam', 'İş-Yaşam'],
            'Teknoloji': ['Teknoloji', 'bilim_ve_teknoloji'],
            'Siyaset': ['siyaset', '2019 Yerel Seçim'],
            'Eğitim': ['Eğitim', 'egitim'],
            'Sağlık': ['Sağlık', 'saglik'],
            'Türkiye': ['Türkiye', 'turkiye'],
            'Röportajlar': ['Röportajlar', 'Özel Röportajlar'],
        }

    category_counts = Counter(dataset['topic'])
    new_data = []

    for item in dataset:
        category = item['topic']
        for new_category, old_categories in groupings.items():
            if category in old_categories:
                category = new_category
                break
        if category_counts[category] < min_samples and category not in groupings.keys():
            category = 'Other'
        new_data.append({'abstract': item['abstract'], 'topic': category})

    new_dataset = {
        'abstract': [item['abstract'] for item in new_data],
        'topic': [item['topic'] for item in new_data]
    }

    new_category_counts = Counter(new_dataset['topic'])
    print("New category distribution:")
    for category, count in new_category_counts.items():
        print(f"{category}: {count}")

    return new_dataset

In [7]:
def get_top_categories(dataset, top_n=10):
    category_counts = Counter(dataset['topic'])
    top_categories = dict(category_counts.most_common(top_n))

    filtered_data = {
        'abstract': [],
        'topic': []
    }

    for abstract, topic in zip(dataset['abstract'], dataset['topic']):
        if topic in top_categories:
            filtered_data['abstract'].append(abstract)
            filtered_data['topic'].append(topic)

    filtered_dataset = Dataset.from_dict(filtered_data)

    print(f"\nTop {top_n} categories and their counts:")
    for category, count in top_categories.items():
        print(f"{category}: {count}")

    return filtered_dataset

In [8]:
# Load and preprocess the dataset
train_data = dataset['train'].select_columns(['abstract', 'topic'])
cleaned_data = clean_and_group_categories(train_data, min_samples=50)
top_categories_dataset = get_top_categories(cleaned_data)

New category distribution:
Fiskos: 1417
Türkiye: 106224
Dünya: 39944
Spor: 30812
Kültür-Sanat: 8712
Gündem: 10423
Yaşam: 16077
Sağlık: 16175
Seyahat: 567
H. Bunu Konuşuyor: 736
Ekonomi: 19603
cevre: 179
Eğitim: 5082
Siyaset: 4704
None: 2719
Teknoloji: 7003
BBC: 775
Merak Edilenler: 194
Emlak: 289
Polemik: 252
Otomobil: 1447
diger: 410
english: 66
Televizyon: 429
Kadına Şiddet: 207
Turizm: 226
Enerji: 241
yerel_yonetimler: 136
Magazin: 482
Sosyal Güvenlik: 172
Other: 790
Medya: 308
Röportajlar: 119
cumhuriyet_ege: 139
Tarifler: 99
Dünyadan: 167
Şipşak: 155
Diğer: 93

Top 10 categories and their counts:
Türkiye: 106224
Dünya: 39944
Spor: 30812
Ekonomi: 19603
Sağlık: 16175
Yaşam: 16077
Gündem: 10423
Kültür-Sanat: 8712
Teknoloji: 7003
Eğitim: 5082


In [9]:
# Categories to remove
categories_to_remove = ["Gündem", "Yaşam"]

filtered_data = {
    'abstract': [],
    'topic': []
}

for abstract, topic in zip(top_categories_dataset['abstract'], top_categories_dataset['topic']):
    if topic not in categories_to_remove:
        filtered_data['abstract'].append(abstract)
        filtered_data['topic'].append(topic)

# Convert to Dataset object
filtered_dataset = Dataset.from_dict(filtered_data)

print("Categories and their counts after removal ('Gundem' and 'Yasam'):")
category_counts = Counter(filtered_dataset['topic'])
for category, count in category_counts.items():
    print(f"{category}: {count}")

Categories and their counts after removal ('Gundem' and 'Yasam'):
Türkiye: 106224
Dünya: 39944
Spor: 30812
Kültür-Sanat: 8712
Sağlık: 16175
Ekonomi: 19603
Eğitim: 5082
Teknoloji: 7003


In [10]:
# Limit the dataset
max_samples = 50000
if len(filtered_dataset) > max_samples:
    filtered_dataset = filtered_dataset.shuffle(seed=42).select(range(max_samples))

print(f"Final dataset size: {len(filtered_dataset)}")

# Use filtered_dataset for further processing
new_dataset = filtered_dataset

Final dataset size: 50000


In [11]:
# Create label mappings
label_list = list(set(new_dataset['topic']))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

print("\nLabel to ID mapping:")
for label, id in label_to_id.items():
    print(f"{label}: {id}")


Label to ID mapping:
Ekonomi: 0
Spor: 1
Kültür-Sanat: 2
Eğitim: 3
Türkiye: 4
Dünya: 5
Teknoloji: 6
Sağlık: 7


In [12]:
# Tokenization function
def tokenize_and_prepare(examples, tokenizer):
    tokenized = tokenizer(examples["abstract"], truncation=True, padding="max_length", max_length=512)  # You might need to increase max_length
    tokenized['labels'] = [label_to_id[label] for label in examples['topic']]
    return tokenized

In [13]:
# Metrics computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [14]:
# Custom Trainer class
class ContiguousTrainer(Trainer):
    def save_model(self, output_dir: Optional[str] = None, _internal_call: bool = False):
        for name, param in self.model.named_parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()
        super().save_model(output_dir, _internal_call)

In [15]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    label_names=["labels"],
    report_to="tensorboard",
    logging_first_step=True,
    dataloader_num_workers=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
# Model names
model_names = [
    "dbmdz/bert-base-turkish-cased",
    "dbmdz/electra-base-turkish-cased-discriminator"
]

In [17]:
# Tokenize datasets (using the first model's tokenizer, as they should be compatible)
tokenizer = AutoTokenizer.from_pretrained(model_names[0])
tokenized_datasets = new_dataset.map(
    lambda examples: tokenize_and_prepare(examples, tokenizer),
    batched=True,
    remove_columns=new_dataset.column_names
)

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [18]:
# Split dataset
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.shuffle(seed=42).select(range(train_size))
eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(train_size, len(tokenized_datasets)))

print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(eval_dataset)}")

Number of training examples: 40000
Number of validation examples: 10000


In [19]:
# Train and evaluate models
for model_name in model_names:
    print(f"\nTraining model: {model_name}")

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))

    # Initialize trainer
    trainer = ContiguousTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation results: {eval_results}")

    # Make predictions
    predictions = trainer.predict(eval_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)

    # Print classification report
    true_labels = [id_to_label[label] for label in eval_dataset['labels']]
    predicted_labels = [id_to_label[pred] for pred in preds]
    print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels))

    # Save the model
    model_save_path = f"/content/drive/MyDrive/news-classifier-models/{model_name.split('/')[-1]}_news_classifier"
    model_to_save = unwrap_model(trainer.model)
    for param in model_to_save.parameters():
        param.data = param.data.contiguous()
    model_to_save.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

print("Training and evaluation completed for all models.")


Training model: dbmdz/bert-base-turkish-cased


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch,Training Loss,Validation Loss,Accuracy
0,0.464800,0.382938,0.859500
2,0.238800,0.321550,0.882700


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [768, 768], strides() = [1, 768]
param.sizes() = [768, 768], strides() = [768, 1] (Triggered internally at ../torch/csrc/autograd/functions/accumulate_grad.h:218.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at ../aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.str

...

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-turkish-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
0,0.892100,0.653892,0.821100
2,0.322100,0.371105,0.872500


Evaluation results: {'eval_loss': 0.37110525369644165, 'eval_accuracy': 0.8725, 'eval_runtime': 18.2864, 'eval_samples_per_second': 546.854, 'eval_steps_per_second': 8.586, 'epoch': 2.9952}

Classification Report:
              precision    recall  f1-score   support

       Dünya       0.88      0.90      0.89      1749
     Ekonomi       0.85      0.81      0.83       855
      Eğitim       0.82      0.83      0.83       235
Kültür-Sanat       0.71      0.70      0.70       402
      Sağlık       0.87      0.89      0.88       637
        Spor       0.96      0.97      0.97      1332
   Teknoloji       0.68      0.54      0.60       323
     Türkiye       0.87      0.88      0.88      4467

    accuracy                           0.87     10000
   macro avg       0.83      0.82      0.82     10000
weighted avg       0.87      0.87      0.87     10000

Training and evaluation completed for all models.
